# 라이브러리 및 데이터 로드

In [ ]:
import pandas as pd
import time
from tqdm import tqdm
from nltk.tokenize import word_tokenize
from konlpy.tag import Mecab
from collections import Counter
from math import  pi
import math
import matplotlib.pyplot as plt
from datetime import datetime
from emoji import core

import re

mecab = Mecab()

import matplotlib.font_manager as fm
from nltk.tokenize import sent_tokenize
from nltk.corpus import gutenberg
from emoji import core
import seaborn as sns
import numpy as np
import nltk
import os
import re

nltk.download('punkt')

In [ ]:
# DataFrame
df_com = pd.read_csv('./data/cafe_comment.csv')
df_main = pd.read_csv('./data/cafe_main.csv')
df_com.drop(['text_number'], axis=1, inplace=True)
df_com['etc'] = '댓글'
df_main.drop(['date','click','text_number'], axis=1, inplace=True)
df_main['etc'] = '본문'
df_com.rename(columns={'comment' : 'text'}, inplace=True)
df = pd.concat([df_com,df_main])
df.sort_values('title')
df.reset_index(drop=True, inplace=True)
df.drop_duplicates(['text'], inplace=True)

display(df.isnull().sum())
df.fillna('_',inplace=True)

display(df)

In [ ]:
# Stop_words
with open('./data/stop_words.txt', encoding='utf-8') as f:
    stop_words = f.readlines()
stop_words = [line.rstrip('\n') for line in stop_words]

# 전처리

## 텍스트 전처리 (+전처리 함수)

In [ ]:
# 태그 제거
def del_tag_check(text):
    text = re.sub('http:', '', text)
    text = re.sub('comhttpsm.', '', text)
    text = re.sub(r'(\d{2,4})-(\d{3,4})-?(\d{0,4})?', '', text) # 전화번호 제거
    text = re.sub('blog.', '', text)
    text = re.sub('naver.', '', text)
    text = re.sub('co. kr', '', text)

    return text

# 표현 전처리
def combine_check(text):

    text = re.sub(r'밀크티초등(?!학)', '밀크티', text)
    text = re.sub('초등\s?밀크티', '밀크티', text)
    text = re.sub('밀크티\s?아이', '밀크티', text)
    text = re.sub(r'아이스크림(?!홈)', '홈런', text)
    # 명사
    # text = re.sub('할미','할머니', text)
    # text = re.sub('티비','텔레비전', text)

    text = re.sub('애','아이', text)
    text = re.sub('ebs','이비에스', text)
    text = re.sub('앨','엘', text)
    text = re.sub('앰','엠', text)
    text = re.sub('전용학습기','패드', text)
    text = re.sub('태블릿','패드', text)
    text = re.sub('테블릿','패드', text)
    text = re.sub('빨간팬','빨간펜', text)
    text = re.sub('온니원','온리원', text)
    text = re.sub('아이켄두','아이캔두', text)
    text = re.sub('와캠','와이즈캠프', text)
    #쿠키? 는 난 안해도 될듯...
    
    text = re.sub('4(세|살)', '유아', text)
    text = re.sub('5(세|살)', '유아', text)
    text = re.sub('6(세|살)', '유아', text)
    text = re.sub('7(세|살)', '유아', text)
    text = re.sub('예비\s?초등', '유아', text)
    text = re.sub('8(세|살)', '초등저', text)
    text = re.sub('9(세|살)', '초등저', text)
    text = re.sub('(초|초등)1', '초등저', text)
    text = re.sub('1학년', '초등저', text)
    text = re.sub('(초|초등)2', '초등저', text)
    text = re.sub('2학년', '초등저', text)
    text = re.sub('(초|초등)3', '초등저', text)
    text = re.sub('3학년', '초등저', text)
    text = re.sub('(초|초등)4', '초등고', text)
    text = re.sub('4학년', '초등고', text)
    text = re.sub('(초|초등)5', '초등고', text)
    text = re.sub('5학년', '초등고', text)
    text = re.sub('(초|초등)6', '초등고', text)
    text = re.sub('6학년', '초등고', text)
    text = re.sub('(초등|초등학교)\s?초등고', '초등고', text)
    text = re.sub('(초등|초등학교)\s?초등저', '초등저', text)

    special_word = ['[*]','o','@', 'x', '0']

    for i in special_word:
        text = re.sub(f'아이{i}두','아이캔두', text)
        text = re.sub(f'아이캔{i}','아이캔두', text)

    return text

# 특수문자 및 초성 전처리 함수

def company_re(text, word, chosung:bool, loc=None): 

# chosung : 앞 두글자만 적어 놓은 글을 정규식으로 정리 할 때 문제가 없나요 ?
# True : 확인, False : 두글자 확인 x (ex: 밀크티, 윙크 : 문제 있음. False 입력)
# loc : chosung이 False 일 때, 앞 두글자 중 정규식을 사용 할 위치 지정 
# 아이(캔두) : 아-ㅇ = 0, 이-ㅇ= 1 이므로 와이(즈캠프) 겹치지 않기 위해 loc=1 입력

# 영어는 모두 소문자화 했다고 가정 (lower())
    if chosung == False and loc is None:
        print(f"'초성 테스트를 False로 입력 시 loc값을 입력해 줘야 합니다. \n chosung = {chosung}, loc = {loc} \n loc은 정규식을 이용할 초성의 위치입니다. (ex. ㅁ크 -> 밀크 : loc = 0")
    elif chosung == False:
        if loc < 0 or loc > 1:
            print(f'loc값이 잘못되었습니다. 입력한 loc 값 : {loc}')

    if len(word) < 2:
        return f'!Error! 검색 단어가 너무 짧습니다. 검색 단어 : {word}'
        
    cho1 = CHOSUNG_extraction(word)[0]
    cho2 = CHOSUNG_extraction(word)[1]

    special_word = ['[*]','o','@', 'x', '0']

    if len(word) == 2:
        if chosung:
            text = re.sub(cho1+cho2, word[:], text)
            text = re.sub(cho1+word[1], word[:], text)
            text = re.sub(word[0]+cho2, word[:], text)

            for i in range(len(word)):
                for re_w in special_word:
                    re_sen = word[:i] + re_w + word[i+1:]

                    text = re.sub(re_sen,word,text)
        
            return text


        else:
            text = re.sub(cho1+cho2, word[:], text)

            if loc == 0:
                text = re.sub(cho1+word[1], word[:], text) # ㅇ이, 아이, text
                for re_w in special_word:
                    re_sen = re_w + word[1]
                    text = re.sub(re_sen, word[:], text)

            elif loc == 1:
                text = re.sub(word[0]+cho2, word[:], text) # 아ㅇ, 아이, text
                for re_w in special_word:
                    re_sen = word[0] + re_w
                    text = re.sub(re_sen, word[:], text)

            return text
        


    elif len(word) > 2:
        if chosung:
            text = re.sub(cho1+cho2, word[0:2], text)
            text = re.sub(cho1+word[1], word[0:2], text)
            text = re.sub(word[0]+cho2, word[0:2], text)
            text = re.sub(rf'{word[0:2]}(?!{word[2]})', word[:], text)
            
            for i in range(len(word)):
                for re_w in special_word:
                    re_sen = word[:i] + re_w + word[i+1:]
                    text = re.sub(re_sen,word,text)

            for i in range(1, len(word)):
                for re_w in special_word:
                    re_sen = word.replace(word[i], re_w).replace(word[i-1], re_w)
                    text = re.sub(re_sen, word, text)

            if len(word) > 3:
                for i in range(len(word)):
                    for re_w in special_word:
                        re_sen = word.replace(word[i], re_w).replace(word[i-1], re_w).replace(word[i-2], re_w)
                        text = re.sub(re_sen, word, text)

            return text


        else:
            text = re.sub(cho1+cho2, word[0:2], text)
            if loc == 0:
                text = re.sub(cho1+word[1], word[0:2], text)
                text = re.sub(rf'{word[0:2]}(?!{word[2]})', word[:], text)

                for i in range(len(word)):
                    for re_w in special_word:
                        re_sen = word[:i] + re_w + word[i+1:]
                        text = re.sub(re_sen,word,text)

                for i in range(1, len(word)):
                    for re_w in special_word:
                        re_sen = word.replace(word[i], re_w).replace(word[i-1], re_w)
                        text = re.sub(re_sen, word, text)

                if len(word) > 3:
                    for i in range(len(word)):
                        for re_w in special_word:
                            re_sen = word.replace(word[i], re_w).replace(word[i-1], re_w).replace(word[i-2], re_w)
                            text = re.sub(re_sen, word, text)

                return text


            elif loc == 1:
                text = re.sub(word[0]+cho2, word[0:2], text)
                text = re.sub(rf'{word[0:2]}(?!{word[2]})', word[:], text)

                for i in range(len(word)):
                    for re_w in special_word:
                        re_sen = word[:i] + re_w + word[i+1:]
                        text = re.sub(re_sen,word,text)

                for i in range(1, len(word)):
                    for re_w in special_word:
                        re_sen = word.replace(word[i], re_w).replace(word[i-1], re_w)
                        text = re.sub(re_sen, word, text)

                if len(word) > 3:
                    for i in range(len(word)):
                        for re_w in special_word:
                            re_sen = word.replace(word[i], re_w).replace(word[i-1], re_w).replace(word[i-2], re_w)
                            text = re.sub(re_sen, word, text)

                return text

# 초성 찾기 함수
def CHOSUNG_extraction(text):

    CHOSUNG_LIST = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 
                    'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 
                    'ㅌ', 'ㅍ', 'ㅎ']
    chosung_str = ''
    for w in list(text):
        if '가'<=w<='힣':
            # 588개 마다 초성이 바뀜.
            chosung_num = (ord(w) - ord('가'))//588
            chosung_str = chosung_str + CHOSUNG_LIST[chosung_num]

        else:
            chosung_str = chosung_str + w
            
    return chosung_str

# 특수문자 제거
def special_check(text):
    # 특수 기호 제거
    text = re.sub('[-=+,#/\?:^$@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》\n★▲;|]',' ', text)

    # '\E' 모양 제거
    text = re.sub('[\a-zA-Z]. ',' ',text)

    # # 이모티콘 제거
    # text = core.replace_emoji(text, replace='')

    # 줄임말 치환
    text = re.sub('와캠', '와이즈', text)
    text = re.sub('와이즈캠프', '와이즈', text)
    text = re.sub('싱크빅', '씽크빅', text)
    text = re.sub('웅진\s?씽크빅', '씽크빅', text)
    text = re.sub('웅진\s?스마트올', '스마트올', text)
    text = re.sub('이캔두', '아이캔두', text)

    # 자/모음만 남은 경우 제거
    text = re.sub('[ㄱ-ㅎ]+', '', text)
    text = re.sub('[ㅏ-ㅣ]+', '', text)
    
    return text

In [ ]:
company_list = ['밀크티', '엘리하이', '엠베스트','윙크', '빨간펜', 
                '온리원', '홈런', '와이즈캠프','스마트올',
                '와캠', '싱크빅', '씽크빅', '웅진', '이캔두']

In [ ]:
df['text'] = df['text'].apply(lambda x : del_tag_check(x))
df['title'] = df['title'].apply(lambda x : del_tag_check(x))

df['text'] = df['text'].apply(lambda x : combine_check(x))
df['title'] = df['title'].apply(lambda x : combine_check(x))

In [ ]:
for company in tqdm(company_list):
    
    # 전처리 예외처리 (*크)
    if company == '밀크티' or company == '윙크':
        df['text'] = df['text'].apply(lambda x : company_re(x, company, False, 0))
        df['title'] = df['title'].apply(lambda x : company_re(x, company, False, 0))

    # elif company == '와이즈캠프' or company == '아이캔두':
    #     df['text'] = df['text'].apply(lambda x : company_re(x, company, False, 1))
    #     df['title'] = df['title'].apply(lambda x : company_re(x, company, False, 1))

    else:
        df['text'] = df['text'].apply(lambda x : company_re(x, company, True))
        df['title'] = df['title'].apply(lambda x : company_re(x, company, True))

df['text'] = df['text'].apply(lambda x : special_check(x))
df['title'] = df['title'].apply(lambda x : special_check(x))

## DateFrame 전처리

In [ ]:
# 키워드 존재 여부
# company_list.append('아이캔두')

for key in tqdm(company_list):
    test_list = []

    for i in range(len(df)):
        if key in df['text'][i] or key in df['title'][i]: # 내용 혹은 제목에 key값이 들어 있는지
            test_list.append(True)

        else:
            test_list.append(False)

    df[key] = test_list # 데이터프레임에 'key' column 생성

In [ ]:
df_sentiment = pd.DataFrame()

for company in company_list:
    df_piece = df[df[company] == True]
    remove = ['board_name', 'cafe_id']
    # remove = list(df_piece.columns)
    # remove.remove('text')
    # remove.remove('title')
    # remove.remove('yyyy')
    # remove.remove('mm')
    df_piece.drop(remove, axis=1, inplace=True)
    df_piece['company'] = company

    df_sentiment = pd.concat([df_sentiment, df_piece], axis=0)

df_sentiment.reset_index(drop=True, inplace=True)
df_sentiment.head()

# 모델 (Sentiment)

In [ ]:
# 긍/부정문 사전 필요

posi_path = "./data/positive_words.txt"
nega_path = "./data/negative_words.txt"

with open(posi_path, encoding='utf-8') as f:
    positive_words = f.readlines()

with open(nega_path, encoding='utf-8') as f:
    negative_words = f.readlines()

positive_words = [line.rstrip('\n') for line in positive_words]
negative_words = [line.rstrip('\n') for line in negative_words]

In [ ]:
# 사전에서 불필요한 단어 없애기 (사전 자체를 미리 수정하면 이 코드 없어도 될듯...)

try:
    pasitive_words = pasitive_words.remove([''])
except:
    pass

try:
    negative_words = negative_words.remove(['저는', '수'])
except:
    pass

In [ ]:
# 주제별 검색어 리스트 (key_dict) ['가격', '품질',' 컨텐츠', '평가', '아이', '학부모']

가격 = '가격'
가격_list = ['비용','가격','교육비','약정','돈','할인','계약','영사','금액','구매','값','위약금','해지','구입','영업','렌트','무료','체험']

품질 = '품질'
품질_list = ['패드', '스마트','기기','온라인','위약금','제품','렌트','키보드','갤럭시','오류','먹통','태블릿','시스템','퀄리티']

컨텐츠 = '컨텐츠'
컨텐츠_list = ['영어','수학','학습지','문제','과학','교재','국어',
        '인강','컨텐츠','사회','개념','한글','한자','영상','미술','파닉스',
        '논술','독해','심화','리딩','게임','프로그램','예체능','콘텐츠','동영상',
        '국사','역사','한국사','도서','자료','발음','원어민','미디어','동화','진단','구구단','난이도',
        '퀄리티','독후','코딩']

평가 = '평가'
평가_list = ['진도','평가','시험','만점','체크','실력','오답','중간','기말','진단','테스트','성적','풀이','채점','정답','등급','수행','경시대회']

아이 = '아이'
아이_list = ['아이','애','공부','학습','습관','우리','첫째','둘째','셋째','본인','초등학생','초등','초딩','중학생','중등','중딩','쌍둥이','자기','혼자','딸','아들']

학부모 = '학부모'
학부모_list = ['엄마','소개','지인','부모','자녀','남편','아빠','어머님','아버님','잔소리','신랑','어른','욕심']

key_dict = {가격:가격_list, 품질:품질_list, 컨텐츠:컨텐츠_list, 평가:평가_list, 아이:아이_list, 학부모:학부모_list}

In [ ]:
all_time_start = time.time()

status = pd.DataFrame(columns=['Company'])
status['Company'] = company_list

for subject in key_dict:
    keyword = key_dict.get(subject)

    print('-----------------------')
    if len(subject) == 2:
        print(f'| 검색 : {subject}          |')
    elif len(subject) == 3:
        print(f'| 검색 : {subject}        |')
    print('-----------------------')

    firm = df_sentiment.get('company')
    text = df_sentiment.get('text')
    text_key = []

    company = {firm[0]:text[0]}
    num = 0
    for i in tqdm(range(len(firm))):
        
        if company.get(firm[i]) != None:
            
            if any(keyword in text[i] for keyword in keyword):

                company[firm[i]] = company.get(firm[i]) + "\n" + text[i]
                text_key.append(text[i])


            else:
                num += 1

        else:
            if any(keyword in text[i] for keyword in keyword):
                company[firm[i]] = text[i]
                text_key.append(text[i])
            else:
                num += 1

    print(f'{len(firm)} 개의 데이터 중 {subject} 데이터 개수 : {len(firm) - num}')

    # 토큰화
    tokens=[]
    for f, t in company.items():
        token=word_tokenize(t)
        tokens.append([f,token])

    # sentiment 점수
    sentiment_firm=[]
    posi_firm=[]
    nega_firm=[]
    round_ = 3
    posi_words = []
    nega_words = []

    start = time.time()

    for token in tokens:
        posi_word = []
        nega_word = []
        firm=token[0]
        sentiment=0
        count=0

        posi = 0
        nega = 0

        for t in tqdm((token[1])):
            if len(t) > 1:
                # if t in positive_words:
                if any(word in t for word in positive_words): # 사전에 morphs를 써서 세분화 분석을 진행해 볼까... 했는데 정확도가 떨어지기 때문에 X
                    posi_word.append(t)
                    sentiment+=1
                    posi += 1
                    count+=1

                    # if count < 20:
                    #     print('테스트중 : ', t)
                    # else : pass
                
                # elif t in negative_words:
                elif any(word in t for word in negative_words):
                    nega_word.append(t)
                    sentiment -=1
                    nega += 1
                    count+=1

            else:
                pass
            #     print(t, '테스트...')
            
            
        
        sentiment_firm.append([firm,round(sentiment/count,round_)])
        posi_firm.append([firm,round(posi/count,round_)])
        nega_firm.append([firm,round(nega/count,round_)])
        posi_words = posi_words + posi_word
        nega_words = nega_words + nega_word
        print(f'{firm} token 개수 : ', len(token[1]))
        print(f'{firm} sentiment filtering 횟수 : ', count)
    sentiment_words = posi_words + nega_words
    end = time.time()

    print('긍정점수 : ', posi_firm)
    print()
    print('부정점수 : ', nega_firm)
    print()
    print('종합점수 : ', sentiment_firm)
    print()
    # print('데이터 수 : ', len(sentiment_words))
    print('걸린 시간 : ', round(end - start, round_), '초')

    # 빈출어

    # all nouns
    nouns = []
    remove_nouns = []
    for i in (sentiment_words):
        nouns = nouns + mecab.nouns(i)

    for i in nouns:
        if len(i) < 2:
            remove_nouns.append(i)

    for i in remove_nouns:
        nouns.remove(i)

    print('all sentiment words : \n', Counter(nouns).most_common(10))

    # positive nouns
    nouns = []
    remove_nouns = []
    for i in (posi_words):
        nouns = nouns + mecab.nouns(i)

    for i in nouns:
        if len(i) < 2:
            remove_nouns.append(i)

    for i in remove_nouns:
        nouns.remove(i)

    print('positive sentiment words : \n', Counter(nouns).most_common(10))

    # negative nouns
    nouns = []
    remove_nouns = []
    for i in (nega_words):
        nouns = nouns + mecab.nouns(i)

    for i in nouns:
        if len(i) < 2:
            remove_nouns.append(i)

    for i in remove_nouns:
        nouns.remove(i)

    print('negative sentiment words : \n', Counter(nouns).most_common(10))

    # # 막대 그래프
    # a=[]
    # for firm in sentiment_firm:
    #     a.append(firm[1]*100)
    # X=np.arange(len(a))

    # plt.title(f"preference for {subject}(%)",fontsize=15)
    # # plt.xticks()
    # plt.ylim(0, 100)
    # plt.xticks([0, 1, 2], labels=['MilkT','Homerun','Wink'])
    # # plt.xlabel('..')
    # bar = plt.bar(X,a, color= ['cornflowerblue','bisque','thistle'], alpha = 1.0, width=0.5)

    # for persent in bar:
    #     height = persent.get_height()
    #     plt.text(persent.get_x() + persent.get_width()/2.0, height, '%.1f' % height, ha='center', va='bottom', size = 10)

    # plt.show()


# 육각형 그리기 준비 (시각화 용 dataframe 형성)
    status_list = []
    for len_key in range(len(company_list)):
        status_list.append(sentiment_firm[len_key][1]*100)
    status[subject] = status_list
    # status[subject] = [sentiment_firm[0][1]*100,sentiment_firm[1][1]*100,sentiment_firm[2][1]*100]

# @@@@ 한글 폰트 깨져서 임시로 넣은 코드... 삭제 예정
status.rename(columns={'가격' : 'price', '품질' : 'quality','컨텐츠' : 'contents', '평가' : 'test', '아이' : 'children', '학부모' : 'parents'}, inplace=True)
status['Company'] = ['MilkT',' Elihigh', 'Mbest', 'Wink', 'Redpen', 'Onlyone', 'Homerun', 'Wisecamp', 'Smartall', 'Reading_gate', 'Icando']

# company 분야별 점수 최대/최소값 구하기

for i in range(len(status)):
    for j in status.columns:
        if status.iloc[i][j] == status.values[i][1:].max():
            # max_option = f'최대값 : {j}({status.iloc[i][j]}점)'
            # # @@@@ 한글 폰트 깨져서 임시로 넣은 코드... 삭제 예정
            max_option = f'max_iter : {j}({round(status.iloc[i][j], round_)})'
        elif status.iloc[i][j] == status.values[i][1:].min():
            # min_option = f'최대값 : {j}({status.iloc[i][j]}점)'
            # # @@@@ 한글 폰트 깨져서 임시로 넣은 코드... 삭제 예정
            min_option = f'min_iter : {j}({round(status.iloc[i][j], round_)})'



# 육각형 모양 그림
## 따로 그리기
labels = status.columns[1:]
num_labels = len(labels)
subplot_len = math.ceil(len(status)/2)
    
angles = [x/float(num_labels)*(2*pi) for x in range(num_labels)] ## 각 등분점
angles += angles[:1] ## 시작점으로 다시 돌아와야하므로 시작점 추가
    
my_palette = plt.cm.get_cmap("Set2", len(status.index))
 
fig = plt.figure(figsize=(15,20))
fig.set_facecolor('white')
 
for i, row in status.iterrows():
    color = my_palette(i)
    data = status.iloc[i].drop('Company').tolist()
    data += data[:1]
    
    ax = plt.subplot(subplot_len,2,i+1, polar=True)
    ax.set_theta_offset(pi / 2) ## 시작점
    ax.set_theta_direction(-1) ## 그려지는 방향 시계방향
    
    plt.xticks(angles[:-1], labels, fontsize=13) ## x축 눈금 라벨
    ax.tick_params(axis='x', which='major', pad=15) ## x축과 눈금 사이에 여백을 준다.
 
    ax.set_rlabel_position(0) ## y축 각도 설정(degree 단위)
    plt.yticks([0,20,40,60,80,100],['0','20','40','60','80','100'], fontsize=10) ## y축 눈금 설정
    plt.ylim(0,100)
    
    ax.plot(angles, data, color=color, linewidth=2, linestyle='solid') ## 레이더 차트 출력
    ax.fill(angles, data, color=color, alpha=0.4) ## 도형 안쪽에 색을 채워준다.
    
    plt.title(row.Company, size=20, color=color,x=-0.2, y=1.2, ha='left') 
    plt.text(-0.9,175,f' {max_option} \n {min_option}')
    # plt.text(-0.8,45,'text_location2')    
 
plt.tight_layout(pad=0) ## subplot간 패딩 조절

from datetime import datetime
now = datetime.now()
# 이미지 저장 (파일이름 : ()YYYYMMhhmm.png)
# 배경 투명 옵션 : transparent = True
plt.savefig(str('MilkT'+now.strftime('%Y%m%d%H%M'))+'.png')
plt.show()

all_time_end = time.time()
print('총 걸린 시간 : ',round(all_time_end - all_time_start, round_), '초')